In [15]:
import numpy as np
import scipy.linalg
from itertools import product

class Simulator:
    def __init__(self, N, H):
        self.N = N  # Number of spins
        self.H = H  # Hamiltonian function

    def classical(self):
        """Simulate the system classically."""
        best_energy = float('inf')
        best_config = None

        # Brute-force over all spin configurations
        for config in range(1 << self.N):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            energy = self.H(spins)
            if energy < best_energy:
                best_energy = energy
                best_config = spins

        return best_config, best_energy

    def quantum(self, T=1.0, shots=1024, num_steps=100):
        """Simulate the system using adiabatic quantum evolution."""
        num_states = 2**self.N

        # Initial state: equal superposition
        state = np.ones(num_states, dtype=complex) / np.sqrt(num_states)

        # Construct the Hamiltonian matrix
        H_initial = np.zeros((num_states, num_states), dtype=complex)
        for i in range(self.N):
            for config in range(num_states):
                flipped_config = config ^ (1 << i)  # Flip the i-th spin
                H_initial[config, flipped_config] = -1  # Pauli-X equivalent

        H_final = np.zeros((num_states, num_states), dtype=complex)
        for config in range(num_states):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            energy = self.H(spins)
            H_final[config, config] = energy

        # Time evolution
        time_points = np.linspace(0, T, num_steps)
        for t in time_points:
            s = t / T
            H_t = (1 - s) * H_initial + s * H_final  # Interpolate between initial and final Hamiltonians
            U = scipy.linalg.expm(-1j * H_t * (T / num_steps))  # Evolution operator for small step
            state = U @ state  # Apply the evolution

        # Compute probabilities of each outcome
        probabilities = np.abs(state)**2
        probabilities /= np.sum(probabilities)  # Normalize probabilities

        # Sample outcomes
        outcomes = np.random.choice(num_states, size=shots, p=probabilities)

        # Map outcomes to bitstrings and compute energies
        def config_to_bitstring(config):
            return bin(config)[2:].zfill(self.N)

        def bitstring_to_spins(bitstring):
            return [1 if bit == '0' else -1 for bit in bitstring]

        counts = {}
        for outcome in outcomes:
            bitstring = config_to_bitstring(outcome)
            counts[bitstring] = counts.get(bitstring, 0) + 1

        # Calculate energies for each observed state
        energies = {
            bitstring: self.H(bitstring_to_spins(bitstring))
            for bitstring in counts
        }

        # Find the minimum-energy configuration
        best_bitstring = min(energies, key=energies.get)
        best_energy = energies[best_bitstring]
        best_spins = bitstring_to_spins(best_bitstring)

        return best_spins, best_energy
    
    def compute_eigenvalues(self):
        """Compute all possible eigenvalues of the Hamiltonian."""
        eigenvalues = set()
        for config in range(1 << self.N):
            spins = [1 if (config & (1 << i)) else -1 for i in range(self.N)]
            eigenvalue = self.H(spins)
            eigenvalues.add(eigenvalue)
        return sorted(eigenvalues)

    def suggest_t(self):
        """Suggest an optimal time t based on the energy gap."""
        eigenvalues = self.compute_eigenvalues()
        if len(eigenvalues) < 2:
            raise ValueError("Insufficient eigenvalues to compute energy gap.")

        # Compute the smallest non-zero energy gap
        gaps = [eigenvalues[i + 1] - eigenvalues[i] for i in range(len(eigenvalues) - 1)]
        delta_E = min(gaps)
        return 1 / delta_E  # Suggest t proportional to the inverse gap

# Example Usage
def sample_hamiltonian(spins):
    """A sample Hamiltonian: Ising model with no external field."""
    return -sum(spins[i] * spins[i + 1] for i in range(len(spins) - 1))

if __name__ == "__main__":
    N = 4  # Number of spins
    sim = Simulator(N, sample_hamiltonian)

    # Classical simulation
    best_classical, energy_classical = sim.classical()
    print("Classical Best Configuration:", best_classical, "Energy:", energy_classical)

    # Quantum simulation
    best_quantum, energy_quantum = sim.quantum(T=1.0, shots=1024)
    print("Quantum Best Configuration:", best_quantum, "Energy:", energy_quantum)


Classical Best Configuration: [-1, -1, -1, -1] Energy: -3
Quantum Best Configuration: [-1, -1, -1, -1] Energy: -3


In [16]:
class NumberPartitioning:
    def __init__(self, numbers):
        self.numbers = numbers
    
    def hamiltonian(self):
        """Define the Ising Hamiltonian for the problem."""
        # H = A * (sum(n_i * s_i))^2
        return lambda spins: (np.sum(np.array(self.numbers) * np.array(spins))) ** 2

# Example Input
numbers = [3, 1, 4, 2, 2]  # Example set of numbers
np_problem = NumberPartitioning(numbers)

sim = Simulator(len(numbers), np_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(T=optimal_t)
print("Quantum Result:", quantum_result)

Classical Result: ([1, 1, -1, 1, -1], 0)
Suggested optimal t: 0.25
Quantum Result: ([1, 1, -1, 1, -1], 0)


In [18]:
class Cliques:
    def __init__(self, edges, k):
        self.edges = edges  # List of tuples representing edges [(u, v), ...]
        self.k = k  # Desired clique size

    def hamiltonian(self):
        """Define the Ising Hamiltonian for the clique problem."""
        def hamiltonian(spins):
            # First term: enforce clique size
            term1 = (self.k - np.sum(spins)) ** 2
            
            # Second term: enforce clique completeness
            term2 = (self.k * (self.k - 1)) / 2 - sum(
                spins[u] * spins[v] for u, v in self.edges
            )
            
            # Combine terms
            A, B = 1.0, 1.0  # Weights for terms
            return A * term1 + B * term2
        
        return hamiltonian
    
# Example Input
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]  # Example graph
num_vertices = 4
k = 3  # Desired clique size
cliques_problem = Cliques(edges, k)

sim = Simulator(num_vertices, cliques_problem.hamiltonian())

# Classical Simulation
classical_result = sim.classical()
print("Classical Result:", classical_result)

# Suggest an optimal t
optimal_t = sim.suggest_t()
print(f"Suggested optimal t: {optimal_t}")

# Quantum Simulation
quantum_result = sim.quantum(T=optimal_t)
print("Quantum Result:", quantum_result)

Classical Result: ([1, 1, 1, 1], -1.0)
Suggested optimal t: 0.5
Quantum Result: ([1, 1, 1, 1], -1.0)
